In [1]:
import math
from pickle import FALSE

import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
from numpy.ma.core import left_shift

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity = 1
)
###################

In [3]:
def detectPose(image, pose, display=True):
    '''
    Perform pose detection on an image.
    Args:
        image: Input image with a prominent person for pose detection.
        pose: Mediapipe pose function for detection.
        display: If True, display the images and landmarks.
    Returns:
        output_image: Image with detected pose landmarks drawn.
        landmarks: List of detected landmarks in original scale.
    '''
    # Create a copy of the input image.
    output_image = image.copy()

    # Convert BGR to RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform Pose Detection.
    results = pose.process(imageRGB)

    # Retrieve the image dimensions.
    height, width, _ = image.shape

    # Initialize list for landmarks.
    landmarks = []

    # If landmarks are detected.
    if results.pose_landmarks:
        # Draw Pose landmarks on the image.
        mp_drawing.draw_landmarks(
            image=output_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS
        )

        # Save the landmarks.
        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x * width), int(landmark.y * height), landmark.z * width))

    # Display results if requested.
    if display:
        plt.figure(figsize=[22, 22])
        plt.subplot(121);
        plt.imshow(image[:, :, ::-1]);
        plt.title("Original Image");
        plt.axis('off');
        plt.subplot(122);
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output Image");
        plt.axis('off');
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        return output_image, landmarks

In [4]:
# Initialize Pose function for video.
pose_video = mp_pose.Pose(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity=1
)

# Open the webcam (change index to 1 if another camera is used).
video = cv2.VideoCapture(0)

# Create named window for resizing.
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

# Set video resolution.
video.set(3, 1280)  # Width
video.set(4, 960)  # Height

# Initialize time for FPS calculation.
time1 = 0

# Video capture loop.
while video.isOpened():
    # Read a frame.
    ok, frame = video.read()

    # If frame is not read properly, break the loop.
    if not ok:
        print("Error: Unable to read frame.")
        break

    # Flip the frame horizontally for natural view.
    frame = cv2.flip(frame, 1)

    # Resize the frame.
    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

    # Perform pose detection.
    try:
        # Convert the frame to RGB for processing.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform pose detection.
        results = pose_video.process(rgb_frame)

        # If landmarks are detected, process them.
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Print and optionally process landmarks.
            for i, landmark in enumerate(landmarks):
                print(
                    f"Landmark {i}: x={landmark.x:.3f}, y={landmark.y:.3f}, z={landmark.z:.3f}, visibility={landmark.visibility:.2f}"
                )

                # Optionally, convert normalized coordinates to pixels.
                pixel_x = int(landmark.x * frame_width)
                pixel_y = int(landmark.y * frame_height)
                print(f"Landmark {i} in pixels: x={pixel_x}, y={pixel_y}")

            # Draw pose landmarks on the frame.
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )

    except Exception as e:
        print("Error during pose detection:", e)
        continue

    # Calculate and display FPS.
    time2 = time()
    if (time2 - time1) > 0:
        fps = 1.0 / (time2 - time1)
        cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    time1 = time2

    # Display the frame.
    cv2.imshow('Pose Detection', frame)

    # Break on 'ESC' key press.
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the video capture object and close windows.
video.release()
cv2.destroyAllWindows()

Landmark 0: x=0.362, y=0.605, z=-1.360, visibility=1.00
Landmark 0 in pixels: x=463, y=435
Landmark 1: x=0.388, y=0.500, z=-1.293, visibility=1.00
Landmark 1 in pixels: x=496, y=360
Landmark 2: x=0.407, y=0.498, z=-1.293, visibility=1.00
Landmark 2 in pixels: x=520, y=358
Landmark 3: x=0.425, y=0.497, z=-1.293, visibility=1.00
Landmark 3 in pixels: x=544, y=357
Landmark 4: x=0.329, y=0.498, z=-1.311, visibility=1.00
Landmark 4 in pixels: x=421, y=358
Landmark 5: x=0.306, y=0.496, z=-1.311, visibility=1.00
Landmark 5 in pixels: x=392, y=357
Landmark 6: x=0.287, y=0.496, z=-1.312, visibility=1.00
Landmark 6 in pixels: x=366, y=357
Landmark 7: x=0.447, y=0.518, z=-0.764, visibility=1.00
Landmark 7 in pixels: x=571, y=373
Landmark 8: x=0.250, y=0.529, z=-0.844, visibility=1.00
Landmark 8 in pixels: x=320, y=381
Landmark 9: x=0.389, y=0.695, z=-1.146, visibility=1.00
Landmark 9 in pixels: x=497, y=500
Landmark 10: x=0.329, y=0.705, z=-1.171, visibility=1.00
Landmark 10 in pixels: x=421, y=5

KeyboardInterrupt: 

In [15]:
for lndmrk in mp_pose.PoseLandmark:
    print(f"{lndmrk.value}: {lndmrk.name}") 

0: NOSE
1: LEFT_EYE_INNER
2: LEFT_EYE
3: LEFT_EYE_OUTER
4: RIGHT_EYE_INNER
5: RIGHT_EYE
6: RIGHT_EYE_OUTER
7: LEFT_EAR
8: RIGHT_EAR
9: MOUTH_LEFT
10: MOUTH_RIGHT
11: LEFT_SHOULDER
12: RIGHT_SHOULDER
13: LEFT_ELBOW
14: RIGHT_ELBOW
15: LEFT_WRIST
16: RIGHT_WRIST
17: LEFT_PINKY
18: RIGHT_PINKY
19: LEFT_INDEX
20: RIGHT_INDEX
21: LEFT_THUMB
22: RIGHT_THUMB
23: LEFT_HIP
24: RIGHT_HIP
25: LEFT_KNEE
26: RIGHT_KNEE
27: LEFT_ANKLE
28: RIGHT_ANKLE
29: LEFT_HEEL
30: RIGHT_HEEL
31: LEFT_FOOT_INDEX
32: RIGHT_FOOT_INDEX


In [12]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.688451707
y: 1.35051668
z: -0.736725926
visibility: 0.173643559

In [13]:
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points using the cosine rule.
    
    Args:
        a: First point (x, y).
        b: Second point (x, y) - the joint point.
        c: Third point (x, y).
    
    Returns:
        angle: Angle in degrees.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # Calculate vectors
    ba = a - b
    bc = c - b

    # Calculate cosine angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    
    # Convert from radians to degrees
    angle = np.degrees(np.arccos(cosine_angle))
    
    return angle

# Extract landmark coordinates
left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame_width, 
                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame_height)

left_elbow = (landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * frame_width, 
              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * frame_height)

left_wrist = (landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * frame_width, 
              landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * frame_height)

# Calculate the angle
elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

print(f"Left Elbow Angle: {elbow_angle:.2f} degrees")



Left Elbow Angle: 29.44 degrees


In [1]:
print("hello world")

hello world
